In [1]:
import pandas as pd
import numpy as np
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g6
g6=rdflib.Graph()
g6.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_Legislature_6.nt',format='nt')
print(len(g6))

101775


In [3]:
#************Change Path to files************
smart_cities_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Smart_Cities.csv") 
smart_cities_data=smart_cities_data.replace(np.nan,"",regex=True)
smart_cities_data.head(5)

,Names of selected cities,Name of State/UT
0,Port Blair,Andaman and Nicobar Island
1,Vishakhapatnam,Andhra Pradesh
2,Tirupati,Andhra Pradesh
3,Kakinada,Andhra Pradesh
4,Pasighat,Arunachal Pradesh


In [4]:
global org
global bhorg
global skos
global bhcatres
global bhorgres
global bhorg

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")

In [5]:
pd.set_option('display.max_colwidth', 500)#housekeeping

In [6]:
#finding n_org max
global n_org
l_lit=[]
for s,p,o in g6.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    l_lit.append(num_i) #list of n_org identifier values
#print(len(l_lit))
l_lit.sort() #sorting the list
n_org=l_lit[-1] #the current maximum value of n_org

#finding n_cat max, 
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g6.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [7]:
print(n_cat)
print(n_org)

46
11004


In [8]:
#fetch_n_rg for org1_an
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g6
    global n_org
    org_id=""
    for s,p,o in g6.triples((None, bhorg.hasName,Literal(org_nam))):
        
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g6.add((org_id, bhorg.hasName, Literal(org_nam)))
        g6.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        if org_url!="":
            g6.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g6.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g6.add((org_id,RDF.type,bhorg.Organisation))
        g6.add((org_id,org.identifier,Literal(n_org)))
        g6.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)
def fetch_n_cat(cat):
    global n_cat
    global g1
    cat_id=""
    for s,p,o in g6.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        #adding relevant triples
        g6.add((cat_id,RDF.type,skos.Concept))
        g6.add((cat_id,skos.prefLabel, Literal(cat)))
        g6.add((cat_id,skos.altLabel, Literal(cat)))
        g6.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

In [9]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g6
    if not (cat2_id, skos.broader,cat1_id) in g6:
        g6.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g6:
        g6.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g6
    if not (org1_id,org.hasSubOrganization,org2_id) in g6:
        g6.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g6:
        g6.add((org2_id,org.subOrganizationOf,org1_id))

In [10]:
#Creating tuple for "Smart City"
scity_id=fetch_n_cat("Smart City")
#scity broader State & UTs, State, UT
state_uts_id=fetch_n_cat("State & Union Territory")
fetch_cat_broader(state_uts_id,scity_id)
state_id=fetch_n_cat("State")
fetch_cat_broader(state_id,scity_id)
ut_id=fetch_n_cat("Union Territory")
fetch_cat_broader(ut_id,scity_id)
nct_id=fetch_n_cat("National Capital Territory")
fetch_cat_broader(nct_id,scity_id)

In [11]:
print(ut_id)
print(nct_id)
print(state_uts_id)

http://semanticwebindia.in/bharathi/res/cat/225
http://semanticwebindia.in/bharathi/res/cat/239
http://semanticwebindia.in/bharathi/res/cat/223


In [12]:
#Creating tuples from smart_cities_data
for i in range(smart_cities_data.shape[0]):
    city_nam=smart_cities_data.iloc[i,0]
    state_ut_nam=smart_cities_data.iloc[i,1]
    #print(city_nam,state_ut_nam)
    rw_scity_id=""
    rw_state_ut_id=""
    for s,p,o in g6.triples((None,bhorg.hasName,Literal(state_ut_nam))):
        for s1,p1,o1 in g6.triples((s,org.classification,None)):
            if o1 in [state_id,ut_id,nct_id]:
                rw_state_ut_id=s
                #print(s,": ",o)
    
    for a,b,c in g6.triples((None,bhorg.hasName,Literal(city_nam))):
        if (a,org.classification,scity_id) in g6:
                rw_scity_id=a
                
    
    if rw_scity_id=="":
        #print("entering if")
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        rw_scity_id=URIRef(bhorgres+str('1')+str(n_org))
        #print(rw_scity_id)
        g6.add((rw_scity_id, bhorg.hasName, Literal(city_nam)))
        g6.add((rw_scity_id, bhorg.hasAlternateName, Literal(city_nam)))
        g6.add((rw_scity_id,RDF.type,bhorg.Organisation))
        g6.add((rw_scity_id,org.identifier,Literal(n_org)))
        g6.add((rw_scity_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
        g6.add((rw_scity_id,org.classification,scity_id))
        fetch_org_subOrg(rw_state_ut_id,rw_scity_id)

In [13]:
len(g6)

102571

In [14]:
for s,p,o in g6.triples((None,bhorg.hasName,Literal("Chandigarh"))):
    for s1,p1,o1 in g6.triples((s,org.classification,None)):
        print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/org/14900 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/230
http://semanticwebindia.in/bharathi/res/org/111014 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/247
http://semanticwebindia.in/bharathi/res/org/14804 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/223
http://semanticwebindia.in/bharathi/res/org/14804 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/225


In [15]:
u=URIRef("http://semanticwebindia.in/bharathi/res/cat/230")
for s,p,o in g6.triples((u,skos.prefLabel,None)):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/cat/230 http://www.w3.org/2004/02/skos/core#prefLabel District


In [16]:
g6.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_SmartCity_7.nt')

In [17]:
smart_cities_data.index

RangeIndex(start=0, stop=98, step=1)

In [18]:
i=0
for s,p,o in g6.triples((None, bhorg.hasName,Literal("Prime Minister's Office"))):
    #print(s,p,o)
    for s1,p1,o1 in g6.triples((s,org.hasSubOrganization,None)):
        for s2,p2,o2 in g6.triples((o1,bhorg.hasName,None)):
            i+=1
            print(i,":",o2)

1 : National Rainfed Area Authority (NRAA)
2 : Tod Fod Jod, National Innovation Council
3 : National Technical Research Organisation (NTRO)
4 : Scientific Advisory Council to the Prime Minister
5 : Mann ki Baat by Hon'ble Prime Minister of India
6 : Economic Advisory Council to The Prime Minister
7 : Direct Benefit Transfer (DBT), MIS - DBT Planning Commission, New Delhi
8 : India Innovation Portal (IIP)
9 : National Innovation Council
10 : National Security Advisory Board (NSAB)
11 : National Critical Information Infrastructure Protection Centre (NCIIPC)
12 : Prime Minister's National Relief Fund, (PMNRF) - Donate Online


In [19]:
for s,p,o in g6.triples((None,bhorg.hasName,Literal("Chandigarh"))):
    for s1,p1,o1 in g6.triples((s,org.classification,None)):
        print(s1,p1,o1)

http://semanticwebindia.in/bharathi/res/org/14900 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/230
http://semanticwebindia.in/bharathi/res/org/111014 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/247
http://semanticwebindia.in/bharathi/res/org/14804 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/223
http://semanticwebindia.in/bharathi/res/org/14804 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/225
